## Adding additional functions to sqlite3

1. install gcc: in case MinGW is not working on windows, an alternative would be [win-builds](http://win-builds.org/doku.php). Install and add path of gcc.exe to PATH environment variable.
2. from [cloudmeters/sqlite](https://github.com/cloudmeter/sqlite) ...download only the file *sqlite3ext.h*
3. Compile as described in the source file with
    > gcc -shared -I "." -o libsqlitefunctions.DLL extension-functions.c
4. locate the file "sqlite3.dll" from your python environment and add *libsqlitefunctions.DLL* to the folder.


## Accessing extended functions in Python

In [4]:
# I) creating an in-memory data base
#    connection by "engine" object
from sqlalchemy import create_engine
sqlfile=":memory:"
engine = create_engine(f"sqlite:///{sqlfile}")

# II) Additional function being called when connecting to db
#     make sure the library "libsqlitefunctions.dll" (windows) is located in the same directory of "sqlite3.dll"
#     ... or, add file-path
from sqlalchemy import event
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_rec):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("libsqlitefunctions")')

# III) open the data base by factory/session object
from sqlalchemy.orm import sessionmaker
session = sessionmaker(bind=engine)()

# IV) create an SQL statement with SQLAlchemy
from sqlalchemy import literal, func
subq=session.query(func.log(10).label("test"))
#subq=session.query(literal(10).label("test"))

# V) show the results with pandas
import pandas as pd
df=pd.read_sql(subq.statement, engine)
print(df)

       test
0  2.302585


## SQL statement

In [2]:
# have a look to the ra SQL
def pretty_sql(q1):
    import sqlparse
    # debug: looking at the SQL pretty printed
    text1=str(q1.statement.compile(engine, compile_kwargs={"literal_binds": True}))
    text2=sqlparse.format(text1, reindent=True, keyword_case='upper')
    return text2

print(pretty_sql(subq))


SELECT log(10) AS test
